In [14]:
import sys
import os
from dotenv import load_dotenv
import pandas as pd

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from sqltoolkit.connectors import PostgreSQLConnector
from sqltoolkit.client import DatabaseClient
from sqltoolkit.entities import Table, TableColumn
from sqltoolkit.compiler import SQLQueryChecker

load_dotenv()

# Define the connection parameters
server = os.getenv('POSTGRES_HOST')
database = 'bird_dev_mini'
username = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PWD')
port = '5432'

psql_connector = PostgreSQLConnector(server, database, username, password, port)
sql_client = DatabaseClient(psql_connector)

from openai import AzureOpenAI

aoai_endpoint = os.getenv('OPENAI_ENDPOINT')
aoai_key = os.getenv('OPENAI_API_KEY')
aoai_deployment = os.getenv('OPENAI_4o_DEPLOYMENT') # should be GPT-4o
aoai_embedding_deployment = os.getenv('OPENAI_EMBEDDING_DEPLOYMENT') # should be text-embedding-small

openai_client = AzureOpenAI(azure_endpoint=aoai_endpoint,
    api_key=aoai_key,
    api_version='2024-10-21')

In [10]:
import os
import pandas as pd

directory = 'documentation'

schema_descriptions =[]
for root, dirs, files in os.walk(directory):
    concatenated_content = ""
    if 'database_description' in dirs:
        db_desc_path = os.path.join(root, 'database_description')
        for filename in os.listdir(db_desc_path):
            if filename.endswith(".csv"):
                table_name = filename.split('.')[0]
                df = pd.read_csv(os.path.join(db_desc_path, filename), encoding='ISO-8859-1')
                table_markdown = df.to_markdown(index=False)
                concatenated_content += f"\n### Table: {table_name}\n\n{table_markdown}\n\n"
        concatenated_content = concatenated_content.encode('utf-8').decode('utf-8')
        schema_descriptions.append({'description':concatenated_content, 'schema':root.split('/')[-1]})
    


In [16]:
schemas

['financial',
 'toxicology',
 'card_games',
 'codebase_community',
 'debit_card_specializing',
 'superhero',
 'thrombosis_prediction',
 'formula_1',
 'california_schools',
 'student_club',
 'european_football_2']

In [17]:
from sqltoolkit.indexer import DatabaseIndexer

search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
search_key = os.getenv('AI_SEARCH_API_KEY')
embedding_deployment = os.getenv('OPENAI_EMBEDDING_DEPLOYMENT')
index_name = "bird-dev-mini"


for schema in schema_descriptions:
    print(f"Indexing schema: {schema['schema']}")
    context = schema['description']
    schema_name = schema['schema']

    indexer = DatabaseIndexer(sql_client, 
                            openai_client, 
                            aoai_deployment,
                            aoai_embedding_deployment,
                            extra_context=context)
    
    manifest = indexer.fetch_and_describe_tables(regex_filter=f"{schema_name}.*")
    indexer.generate_table_embeddings()

    indexer.create_azure_ai_search_index(
    search_endpoint=search_endpoint,
    search_credential=search_key,
    index_name=index_name,
    embedding_deployment=aoai_embedding_deployment,
    openai_endpoint=aoai_endpoint,
    openai_key=aoai_key,
)

# write to AI Search
    indexer.push_to_ai_search()


2025-01-23 19:00:20,790 - DatabaseIndexer - INFO - Fetching tables from the database.


Indexing schema: financial


2025-01-23 19:00:21,057 - DatabaseIndexer - INFO - Filtered tables by regex: ['financial.order', 'financial.loan', 'financial.account', 'financial.card', 'financial.client', 'financial.disp', 'financial.district', 'financial.trans']
2025-01-23 19:00:21,058 - DatabaseIndexer - INFO - Processing table: financial.order
2025-01-23 19:00:39,044 - DatabaseIndexer - INFO - Completed processing table: financial.order
2025-01-23 19:00:39,046 - DatabaseIndexer - INFO - Processing table: financial.loan


Object of type date is not JSON serializable


2025-01-23 19:01:01,900 - DatabaseIndexer - INFO - Completed processing table: financial.loan
2025-01-23 19:01:01,903 - DatabaseIndexer - INFO - Processing table: financial.account


...

2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,535 - DatabaseIndexer - INFO - Data for table formula_1.status pushed to the index.
2025-01-23 19:20:12,631 - DatabaseIndexer - INFO - Data for table formula_1.results pushed to the index.
2025-01-23 19:20:12,631 - DatabaseIndexer - INFO - Data for tab

Pushing data for table formula_1.results to the index.
Indexing schema: california_schools


2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,231 - DatabaseIndexer - INFO - Completed processing table: california_schools.frpm
2025-01-23 19:23:29,244 - DatabaseIndexer - INFO - Processing table: cali

Object of type date is not JSON serializable
Object of type date is not JSON serializable
Object of type date is not JSON serializable


KeyboardInterrupt: 